In [6]:
import pandas as pd
import requests

from bs4 import BeautifulSoup
from io import StringIO

In [2]:
# Data as a string
data = """ID,Team,Abbreviation,Conference,Division
1,Arizona Cardinals,ARI,NFC,West
2,Atlanta Falcons,ATL,NFC,South
3,Baltimore Ravens,BAL,AFC,North
4,Buffalo Bills,BUF,AFC,East
5,Carolina Panthers,CAR,NFC,South
6,Chicago Bears,CHI,NFC,North
7,Cincinnati Bengals,CIN,AFC,North
8,Cleveland Browns,CLE,AFC,North
9,Dallas Cowboys,DAL,NFC,East
10,Denver Broncos,DEN,AFC,West
11,Detroit Lions,DET,NFC,North
12,Green Bay Packers,GB,NFC,North
13,Houston Texans,HOU,AFC,South
14,Indianapolis Colts,IND,AFC,South
15,Jacksonville Jaguars,JAX,AFC,South
16,Kansas City Chiefs,KC,AFC,West
17,Miami Dolphins,MIA,AFC,East
18,Minnesota Vikings,MIN,NFC,North
19,New England Patriots,NE,AFC,East
20,New Orleans Saints,NO,NFC,South
21,New York Giants,NYG,NFC,East
22,New York Jets,NYJ,AFC,East
23,Las Vegas Raiders,LV,AFC,West
24,Philadelphia Eagles,PHI,NFC,East
25,Pittsburgh Steelers,PIT,AFC,North
26,Los Angeles Chargers,LAC,AFC,West
27,San Francisco 49ers,SF,NFC,West
28,Seattle Seahawks,SEA,NFC,West
29,Los Angeles Rams,LAR,NFC,West
30,Tampa Bay Buccaneers,TB,NFC,South
31,Tennessee Titans,TEN,AFC,South
32,Washington Commanders,WAS,NFC,East"""

# Convert the string data into a pandas DataFrame
nfl_teams_df = pd.read_csv(StringIO(data))

In [3]:
# URL for the end of season 2023 power rankings
url = 'https://www.espn.com/nfl/story/_/page/NFLpowerranking010224/nfl-week-18-power-rankings-poll-2023-plus-biggest-contract-offseason-situations'

# Set headers to mimic a browser
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Send a request to fetch the content of the page
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

# Find the div with class 'article-body'
article_body = soup.find('div', class_='article-body')

# Placeholder for data
rankings_data = []

# Adjust selectors based on the actual HTML structure
for header in article_body.find_all('h2'):
    text = header.text.strip()
    parts = text.split('.', 1)
    try:
        rank, team_info = parts[0].strip(), parts[1]
        if not rank.isdigit():
            continue
        name, _ =  team_info.split(' (')
        rankings_data.append((rank, name))
    except:
        continue

# Create a DataFrame
prev_rankings_df = pd.DataFrame(rankings_data, columns=['Rank', 'Team'])

In [4]:
# URL for the preseason 2024 power rankings
url = 'https://www.espn.com/nfl/story/_/id/40884348/nfl-power-rankings-preseason-poll-2024-hot-seat-candidates-players-coaches-execs'

# Send a request to fetch the content of the page
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

# Send a request to fetch the content of the page
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

# Find the div with class 'article-body'
article_body = soup.find('div', class_='article-body')

# Placeholder for data
rankings_data = []

# Adjust selectors based on the actual HTML structure
for header in article_body.find_all('h2'):
    text = header.text.strip()
    parts = text.split('.', 1)
    try:
        rank, name = parts[0].strip(), parts[1]
        if not rank.isdigit():
            continue
        rankings_data.append((rank, name))
    except:
        continue

# Create a DataFrame
curr_rankings_df = pd.DataFrame(rankings_data, columns=['Rank', 'Team'])

In [5]:
# Strip whitespace and ensure consistent naming
nfl_teams_df['Team'] = nfl_teams_df['Team'].str.strip()
curr_rankings_df['Team'] = curr_rankings_df['Team'].str.strip()
prev_rankings_df['Team'] = prev_rankings_df['Team'].str.strip()

# Merge the ranking DataFrames
merged_df = pd.merge(prev_rankings_df, curr_rankings_df, on='Team', how='outer', suffixes=('_2023', '_2024'))

# Merge with the original team DataFrame
final_df = pd.merge(nfl_teams_df, merged_df, left_on='Team', right_on='Team', how='left')

# Save the DataFrame to a CSV file after changing a couple column names
final_df.rename(columns={'Rank_2023': 'Rank_EOS_2023', 'Rank_2024': 'Rank_Pre_2024'}, inplace=True)
final_df.to_csv('nfl_teams.csv', index=False)